# Evaluate Retrieval Augmented Generation with Anthropic Claude 3, Amazon Bedrock, Langchain and RAGAS

## Introduction

In this notebook we will show you how to use Langchain, Anthropic Claude 3, Knowledge base for Amazon Bedrock and RAGAS to evaluate response of a Retrieval Augmented Generation (RAG) solution


#### Use case

Evaluation of RAG (Retrieval-Augmented Generation) Application for Private Documentation Question Answering 


#### Persona
As an analyst at Anycompany, Bob wants to evaluate the response of a Retrieval-Augmented Generation (RAG) application for answering questions based on the company's private documentation. The RAG application combines information retrieval and language generation techniques to provide accurate and relevant answers to users' questions. 

#### Implementation
To fulfill this use case, in this notebook we will show how to evaluate a RAG Application to answer questions from business data. We will use the Anthropic Claude 3 Sonnet Foundation model, Knowledge base for Amazon Bedrock, Langchain and RAGAS. 

#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore and langchain.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain>0.1 --force-reinstall --quiet
%pip install ragas>0.1 --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.bedrock import BedrockChat
from botocore.client import Config
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
from langchain.schema.runnable import RunnablePassthrough
from datasets import Dataset
from langchain.embeddings import BedrockEmbeddings
import pandas as pd
from langchain.chains import RetrievalQA

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [ ]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider

llm = BedrockChat(model_id=modelId, client=bedrock_client)

## Retrieval

Enter Knowledge Base id and retrieve relevant documents from Knowledge Base for Amazon Bedrock

In [ ]:
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="<knowledge_base_id>", # enter knowledge base id here
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
)

## Model Invocation and Response Generation using RetrievalQA chain

Invoke the model and visualize the response

In [ ]:
query = "Which was the first inference chip launched by AWS according to the passage?"

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=retriever, return_source_documents=True
)

response = qa_chain.invoke(query)
print(response["result"])

## Preparing the Evaluation Data

As RAGAS aims to be a reference-free evaluation framework, the required preparations of the evaluation dataset are minimal. You will need to prepare `question` and `ground_truths` pairs from which you can prepare the remaining information through inference as shown below. If you are not interested in the `context_recall` metric, you don’t need to provide the `ground_truths` information. In this case, all you need to prepare are the questions.

In [ ]:
questions = ["How many days has Amazon asked employees to come to work in office?", 
             "By what percentage did AWS revenue grow year-over-year in 2022?",
             "Compared to Graviton2 processors, what performance improvement did Graviton3 chips deliver according to the passage?",
             "Which was the first inference chip launched by AWS according to the passage?",
             "According to the context, in what year did Amazon's annual revenue increase from $245B to $434B?"
            ]
ground_truths = [["Amazon has asked corporate employees to come back to office at least three days a week beginning May 2022."],
                ["AWS had a 29% year-over-year ('YoY') revenue in 2022 on $62B revenue base."],
                ["In 2022, AWS delivered their Graviton3 chips, providing 25% better performance than the Graviton2 processors."],
                ["AWS launched their first inference chips (“Inferentia”) in 2019, and they have saved companies like Amazon over a hundred million dollars in capital expense."],
                ["Amazon's annual revenue increased from $245B in 2019 to $434B in 2022."]]

answers = []
contexts = []

for query in questions:
  answers.append(qa_chain.invoke(query)["result"])
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

## Evaluating the RAG application

First, import all the metrics you want to use from `ragas.metrics`. Then, you can use the `evaluate()` function and simply pass in the relevant metrics and the prepared dataset. Below is a brief description of the metrics

* **Faithfulness**: This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
* **Answer Relevance**: The evaluation metric, Answer Relevancy, focuses on assessing how pertinent the generated answer is to the given prompt. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy. This metric is computed using the question, the context and the answer. Please note, that eventhough in practice the score will range between 0 and 1 most of the time, this is not mathematically guaranteed, due to the nature of the cosine similarity ranging from -1 to 1.
* **Context Precision**: Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.
* **Context Recall**: Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.
* **Context entities recall**: This metric gives the measure of recall of the retrieved context, based on the number of entities present in both ground_truths and contexts relative to the number of entities present in the ground_truths alone. Simply put, it is a measure of what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc. This metric can help evaluate the retrieval mechanism for entities, based on comparison with entities present in ground_truths, because in cases where entities matter, we need the contexts which cover them.
* **Answer Semantic Similarity**: The concept of Answer Semantic Similarity pertains to the assessment of the semantic resemblance between the generated answer and the ground truth. This evaluation is based on the ground truth and the answer, with values falling within the range of 0 to 1. A higher score signifies a better alignment between the generated answer and the ground truth.
* **Answer Correctness**: The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness. Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a ‘threshold’ value to round the resulting score to binary, if desired.
* **Aspect Critique**: This is designed to assess submissions based on predefined aspects such as harmlessness and correctness. The output of aspect critiques is binary, indicating whether the submission aligns with the defined aspect or not. This evaluation is performed using the ‘answer’ as input.

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)
from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

#You can also choose a different model for evaluation
llm_for_evaluation = BedrockChat(model_id=modelId, client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",client=bedrock_client)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

df = result.to_pandas()


pd.options.display.max_colwidth = 800
df

## Conclusion
You have now experimented with `RAGAS` SDK to evaluate a RAG Application using Anthropic Claude 3 as judge

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You